# Import the libraries that are needed

In [ ]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
from tensorflow import keras
from tensorflow.keras import layers
import sklearn
from sklearn import preprocessing
import scipy as sc
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer

#initialize the train and test data set + perform the fourier transformation

In [ ]:
targetdata =  #data that we want to predict
dim_targetdata = #number of values that need to be predicted per input point
features = #input data for the model
dim_modinfo = #number of input parameters per input point
train_size = #number of datapoints in the train dataset
test_size = #number of datapoints in the test dataset
dim_latent_space = #dimension of the latent space representation
num_feature_maps = #number of feature maps in the convolutional layer used to predict the parameters (optimization might yield different parameters for the fouriercomponents and the hidden states but that is not taken into account here)
kernel_size = #dimension of the kernel in the convolutional layer used to predict the parameters(kernel_size x kernel_size) (optimization might yield different parameters for the fouriercomponents and the hidden states but that is not taken into account here)
dense_layer_size = #number of units for the dense layer (optimization might yield different parameters for the fouriercomponents and the hidden states but that is not taken into account here)
grids = #input figures
dimgrids = #dimensionality of the grids

In [ ]:
#perform the fourier transform
verschilfour = []
for ii in range(len(targetdata)):
  verschilfour.append(sc.fft.fft(targetdata[ii]))
verschilfour = np.array(verschilfour)
#split in real and imaginary part
re = np.real(verschilfour)
im = np.imag(verschilfour)
topred = []
for ii in range(len(re)):
  tijdsding = []
  for jj in range(len(re[0])):
    elnt = [re[ii,jj],im[ii,jj]]
    tijdsding.append(elnt)
  topred.append(tijdsding)
topred = np.array(topred)

indexen = range(len(verschilfour))

#split in train and test data set
X_trainges, X_testges, indexen_trainges_final, indexen_testges_final = train_test_split(features, indexen, train_size=train_size, test_size=test_size, random_state=333)
training_yges = topred[indexen_trainges_final]
y_testges = topred[indexen_testges_final]

Exploit symmetry of the fouriertransform

In [ ]:
redeel = (topred[:,:,0]).copy()
for ii in range(int((dim_targetdata-1)/2),dim_targetdata):
  redeel[:,ii] = topred[:,int((dim_targetdata+1)/2-abs((dim_targetdata-1)/2-ii)),0]

In [ ]:
imdeel = (topred[:,:,1]).copy()
for ii in range(int((dim_targetdata-1)/2),dim_targetdata):
  imdeel[:,ii] = -topred[:,int((dim_targetdata+1)/2-abs((dim_targetdata-1)/2-ii)),1]

# Define the fouriercomponents that will be used for the prediction

In [ ]:
relim = #[x,y,z]: fouriercomponents from 0 to x and from y to z are taken into account for the real part with the last ones mirrored from the first ones
imlim = #[x,y,z]: fouriercomponents from 0 to x and from y to z are taken into account for the imag part with the last ones mirrored from the first ones

In [ ]:
predfour = topred.copy()
for jj in range(len(predfour)):
  for ii in range(imlim[0],imlim[1]):
    predfour[jj][ii][1] = 0
  for ii in range(relim[0],relim[1]):
    predfour[jj][ii][0] = 0
  for ii in range(imlim[1],imlim[2]):
    predfour[jj][ii][1] = -topred[jj,np.int((dim_targetdata+1)/2-abs((dim_targetdata-1)/2-ii)),1]
  for ii in range(relim[1],relim[2]):
    predfour[jj][ii][0] = topred[jj,np.int((dim_targetdata+1)/2-abs((dim_targetdata-1)/2-ii)),0]  

Prediction based on the limited number of fourier components

In [ ]:
pred = []
for ii in range(len(predfour)):
  tijdsvector = []
  for jj in range(len(predfour[0])):
    tijdsvector.append(complex(predfour[ii,jj,0],predfour[ii,jj,1]))
  pred.append(tijdsvector)
predictie = np.real(sc.fft.ifft(pred))

# Use neural network to estimate the difference between the fourierprediction and the real values

In [ ]:
errfour = []
for ii in range(len(targetdata)):
  errfour.append(targetdata[ii]-predictie[ii])

split in train and test dataset

In [ ]:
indexenfourr = range(len(errfour))
errfour_train, errfour_test, indexen_trainges, indexen_testges = train_test_split(errfour, indexenfourr, train_size=train_size, test_size=test_size, random_state=333)

make and train the neural network

In [ ]:
X = np.array(errfour_train).reshape(len(errfour_train), dim_targetdata)
params_input = keras.Input(shape=(dim_targetdata))
encoder = tf.keras.layers.Dense(dim_latent_space,activation = 'tanh')(params_input)
decoder = tf.keras.layers.Dense(dim_targetdata)(encoder)
modelauto = keras.Model(inputs=params_input,outputs=decoder)
modelauto.compile(optimizer='adam', loss='mse')
modelauto.summary()
hist = modelauto.fit(X,X,epochs=500,validation_split=0.2,batch_size=32)
plt.semilogy(hist.history['loss'])

Prediction based on limited number of fourier components and predicted difference

In [ ]:
test_input = np.array(errfour).reshape(len(errfour), dim_targetdata)
test_output = np.array(modelauto.predict(test_input, verbose=0)) # = echt - fourier

In [ ]:
uiteindelijkeschatting = test_output + predictie

# Prediction of the parameters

In [ ]:
fourtopred = np.concatenate([predfour[:,0:5,0],predfour[:,1:4,1]],axis = 1) # 1:4 because first one is always 0
#obtain the hidden vectors
Xen = np.array(errfour).reshape(len(errfour), dim_targetdata)
layer_output = modelauto.get_layer('dense').output #dense can need an extra number, depending on how much the algorithm is performed
intermediate_model=tf.keras.models.Model(inputs=params_input,outputs=layer_output)
tussen=intermediate_model.predict(Xen) # = hidden vectors!
tevoorspellen = np.concatenate([fourtopred,tussen],axis = 1)

In [ ]:
indexen = range(len(features))
X_train_four, X_test_four, indexen_train_four, indexen_test_four = train_test_split(grids, indexen, train_size=train_size, test_size=test_size, random_state=333)
training_y_four = fourtopred[indexen_train_four]
y_test_four = fourtopred[indexen_test_four]

X = np.array(X_train_four).reshape(len(X_train_four), dimgrids,dimgrids,1)

Y = np.array(training_y_four).reshape(len(training_y_four), 8)
#using a standardscaler here is advised

Make and train the neural network

In [ ]:
modelsfour = Sequential()
modelsfour.add(Conv2D(num_feature_maps,kernel_size=kernel_size,activation='relu',input_shape=(dimgrids, dimgrids, 1)))
modelsfour.add(MaxPooling2D(2, 2))
modelsfour.add(Flatten())
modelsfour.add(Dense(dense_layer_size, activation='relu'))
modelsfour.add(Dropout(0.2))
modelsfour.add(Dense(8))
modelsfour.compile(optimizer="adam", loss="mse", metrics=["accuracy"])
modelsfour.summary()
hist = modelfour.fit(X,Y,epochs=1000,validation_split=0.2,batch_size=32) 

In [ ]:
test_input = np.array(grids).reshape(len(grids), dimgrids,dimgrids,1)
test_output = np.array(modelfour.predict(test_input, verbose=0))
fourpredictienn = np.vstack(np.transpose(np.array((test_output[:,0],test_output[:,1],test_output[:,2],test_output[:,3],test_output[:,4],np.zeros(len(test_output)),test_output[:,5],test_output[:,6],test_output[:,7]))))

##Predict the hidden state

In [ ]:
indexen = range(len(features))
feathid = []
#add the predicted fouriercomponents to the input of hiddenstate prediction
for ii in range(len(features)):
  feathid.append(np.concatenate([features[ii], fourpredictienn[ii]]))


X_train_hid, X_test_hid, indexen_train_hid, indexen_test_hid = train_test_split(grids, indexen, train_size= train_size, test_size=test_size, random_state=333)
training_y_hid = tussen[indexen_train_hid]
y_test_hid = tussen[indexen_test_hid]
X2_train = fourpredictienn[indexen_train_hid]
X2_test = fourpredictienn[indexen_test_hid]

X1 = np.array(X_train_hid).reshape(len(X_train_hid), dimgrids,dimgrids,1)
X2 = np.array(X2_train).reshape(len(X2_train),9)

Y = np.array(training_y_hid).reshape(len(training_y_hid), 5)
#again it's better to standardscale

In [ ]:
inp1 = keras.Input(shape=(dimgrids,dimgrids,1))
inp2 = keras.Input(shape=(9))
convlayer = Conv2D(num_feature_maps,kernel_size=kernel_size,activation='relu',input_shape=(dimgrids, dimgrids, 1))(inp1)
maxpool = MaxPooling2D(2,2)(convlayer)
flatlay = Flatten()(maxpool)
denselay = Dense(dense_layer_size, activation='relu')(flatlay)
droplay = Dropout(0.2)(denselay)
conclay = layers.concatenate([droplay,inp2])
denselas = (Dense(5))(conclay)
modelhid = keras.Model(inputs=(inp1,inp2),outputs=denselas)
modelhid.compile(optimizer="adam", loss="mse", metrics=["accuracy"])
modelhid.summary()

In [ ]:
hist = modelhid.fit((X1,X2),Y,epochs=500,validation_split=0.2,batch_size=32)

In [ ]:
xtetest = grids
x2tetest = fourpredictienn
test_input = np.array(xtetest).reshape(len(xtetest), dimgrids,dimgrids,1)
test2_input = np.array(x2tetest).reshape(len(x2tetest),9)
test_output = np.array(modelhid.predict((test_input,test2_input), verbose=0))
hidpredictienn = transtarg.inverse_transform(test_output)

# From estimated parameters back to prediction

In [ ]:
fourpred = fourpredictienn
hiddenpred = hidpredictienn

In [ ]:
allefourpredtussen = []
for jj in range(len(topred)):
  toetevoegen = np.zeros((99,2))
  for ii in range(imlim[0],imlim[1]):
    toetevoegen[ii][1] = 0
  for ii in range(relim[0],relim[1]):
    toetevoegen[ii][0] = 0
  for ii in range(0,relim[0]):
    toetevoegen[ii][0] = fourpred[jj,ii]
  for ii in range(0,imlim[0]):
    toetevoegen[ii][1] = fourpred[jj,ii+5]
  allefourpredtussen.append(toetevoegen)

allefourpred = allefourpredtussen.copy()
for jj in range(len(topred)):
  for ii in range(imlim[1],imlim[2]):
    allefourpred[jj][ii][1] = -allefourpredtussen[jj][np.int((dim_targetdata+1)/2-abs((dim_targetdata-1)/2-ii))][1]
  for ii in range(relim[1],relim[2]):
    allefourpred[jj][ii][0] = allefourpredtussen[jj][np.int((dim_targetdata+1)/2-abs((dim_targetdata-1)/2-ii))][0]  

In [ ]:
allefourpredcompl = []
for jj in range(len(allefourpred)):
  tijdsvector = []
  for ii in range(len(allefourpred[0])):
      tijdsvector.append(complex(allefourpred[jj][ii][0],allefourpred[jj][ii][1]))
  allefourpredcompl.append(tijdsvector)

Inverse fouriertransform

In [ ]:
tijdslijntjenafour = np.real(sc.fft.ifft(allefourpredcompl))

Adding the auto-encoder

In [ ]:
Xen = np.array(hiddenpred).reshape(len(hiddenpred), dim_latent_space)
layer_output = modelauto.get_layer('dense_1').output #last laag
layer_input = modelauto.get_layer('dense').output #first dense layer
automoduit =tf.keras.models.Model(inputs=layer_input,outputs=layer_output)
correctiepred=automoduit.predict(Xen) # = hidden vectors!

In [ ]:
finalepred = correctiepred + tijdslijntjenafour